In [560]:
import os
from os import environ

import requests
import io
import string

import datetime

import numpy as np
import pandas as pd

from pandas import Series
from pandas import DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [561]:
cluster_variables=['Route','Flight Number','Weekday','SEASON']

In [562]:
Raw=pd.read_csv('JAN2017_APR2019_R.csv',sep=';')

# Raw=pd.concat([pd.read_csv('JAN2017_DEC2017.csv',sep=';'),
# pd.read_csv('JAN2018_DEC2018.csv',sep=';'),
# pd.read_csv('JAN2019_APR2019.csv',sep=';')])

Raw['Route']=Raw['Sched Airport Depart']+'-'+Raw['Sched Airport Arriva']
Raw=Raw.loc[Raw['Load Factor'].apply(lambda x: x.replace(',','').isnumeric()),:]

Raw=Raw.rename(columns={'Coupons+YQ in current bookings': 'Revenue'})

Raw['Revenue']=Raw['Revenue'].apply(lambda x: np.float(x.replace('EUR','').replace(',','.').replace(' ','')))
Raw=Raw.rename(columns={'Revenue': 'Revenue_actual'})

In [563]:
Raw['Calendar day']=Raw['Calendar day'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))
Raw['LF_actual']=Raw['Load Factor'].apply(lambda x: float(x.replace(',','.').replace('%',''))/100)
Raw['LF_actual']=Raw['Load Factor'].apply(lambda x: float(x.replace(',','.').replace('%','')))
Raw['Direction']=Raw[['Sched Airport Depart','Sched Airport Arriva']].apply(lambda x: 'O' if x[0]=='LUX' and (x[1]=='CPH' or x[1]=='LCY') else 'I',axis=1)

Raw=Raw.rename(columns={'Calendar day': 'Departure Date'})

In [564]:
Raw=Raw.loc[Raw['Sched Airport Depart'].apply(lambda x: (x=='CPH')|(x=='LCY')|(x=='LUX'))&\
Raw['Sched Airport Arriva'].apply(lambda x: (x=='CPH')|(x=='LCY')|(x=='LUX')),:]

In [565]:
Raw=Raw.loc[Raw['Departure Date']>=pd.to_datetime('2018-04-01'),:]

### Seasonality Clustering

In [566]:
Raw['SEASON']='M'

In [567]:
# LUX-CPH

    ## LOW

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-31'))&(Raw['Departure Date']<=pd.to_datetime('2017-08-11')),'SEASON']='L'    

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-31'))&(Raw['Departure Date']<=pd.to_datetime('2017-08-11')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-08-14'))&(Raw['Departure Date']<=pd.to_datetime('2017-09-17')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-19'))&(Raw['Departure Date']<=pd.to_datetime('2017-12-22')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-31'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-17')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-27'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-05'))&(Raw['Departure Date']<=pd.to_datetime('2017-04-08')),'SEASON']='H'    

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-28'))&(Raw['Departure Date']<=pd.to_datetime('2017-04-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-05-23'))&(Raw['Departure Date']<=pd.to_datetime('2017-05-25')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-01'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-05')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-22'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-24')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-13'))&(Raw['Departure Date']<=pd.to_datetime('2017-07-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-08-12'))&(Raw['Departure Date']<=pd.to_datetime('2017-08-13')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-10-27'))&(Raw['Departure Date']<=pd.to_datetime('2017-10-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-15'))&(Raw['Departure Date']<=pd.to_datetime('2017-12-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-09'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-11')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']==pd.to_datetime('2018-04-01')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-27'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-29')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-08'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-10')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-21')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-13'))&(Raw['Departure Date']<=pd.to_datetime('2018-07-18')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-11'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-12')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-10-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-10-28')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-20'))&(Raw['Departure Date']<=pd.to_datetime('2018-12-23')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2019-02-15'))&(Raw['Departure Date']<=pd.to_datetime('2019-02-17')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-05'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-07')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='CPH')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-29'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-30')),'SEASON']='H'

In [568]:
# CPH-LUX

    ## HIGH

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2017-01-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-01-06'))&(Raw['Departure Date']<=pd.to_datetime('2017-01-08')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-11-04'))&(Raw['Departure Date']<=pd.to_datetime('2017-11-06')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-03')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-02')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-08-15'))&(Raw['Departure Date']<=pd.to_datetime('2018-08-16')),'SEASON']='H'

Raw.loc[(Raw['Sched Airport Depart']=='CPH')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-07')),'SEASON']='H'

In [569]:
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY'),'SEASON']='H'
Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX'),'SEASON']='H'

In [570]:
# LUX-LCY

    ## LOW

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2017-01-08')),'SEASON']='L'    
 
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-02-18'))&(Raw['Departure Date']<=pd.to_datetime('2017-02-26')),'SEASON']='L'      
 
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-08'))&(Raw['Departure Date']<=pd.to_datetime('2017-04-23')),'SEASON']='L'      
 
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-28'))&(Raw['Departure Date']<=pd.to_datetime('2017-05-01')),'SEASON']='L'      
 
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-05-25'))&(Raw['Departure Date']<=pd.to_datetime('2017-05-26')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-05'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-11')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']==pd.to_datetime('2017-06-23')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-15'))&(Raw['Departure Date']<=pd.to_datetime('2017-09-03')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-10-30'))&(Raw['Departure Date']<=pd.to_datetime('2017-11-01')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-11-04'))&(Raw['Departure Date']<=pd.to_datetime('2017-11-05')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-13')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-18')),'SEASON']='L' 
   
Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-03')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-07'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-10')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-28'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-01')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-27')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-06'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-07')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LUX')&(Raw['Sched Airport Arriva']=='LCY')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-13'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-22')),'SEASON']='L'

In [571]:
# LCY-LUX

    ## LOW

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-01-01'))&(Raw['Departure Date']<=pd.to_datetime('2017-01-08')),'SEASON']='L'    

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-02-18'))&(Raw['Departure Date']<=pd.to_datetime('2017-02-26')),'SEASON']='L'    
   
Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-04-08'))&(Raw['Departure Date']<=pd.to_datetime('2017-04-23')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-05-01')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-05-25'))&(Raw['Departure Date']<=pd.to_datetime('2017-05-28')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-03'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-11')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-06-23'))&(Raw['Departure Date']<=pd.to_datetime('2017-06-25')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-07-15'))&(Raw['Departure Date']<=pd.to_datetime('2017-09-03')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-10-28'))&(Raw['Departure Date']<=pd.to_datetime('2017-11-01')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-11-04'))&(Raw['Departure Date']<=pd.to_datetime('2017-11-05')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2017-12-18'))&(Raw['Departure Date']<=pd.to_datetime('2018-01-07')),'SEASON']='L' 

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-02-17'))&(Raw['Departure Date']<=pd.to_datetime('2018-02-18')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-04-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-04-15')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']==pd.to_datetime('2018-05-01')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-10'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-13')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-19'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-21')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-05-26'))&(Raw['Departure Date']<=pd.to_datetime('2018-05-27')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-07-14'))&(Raw['Departure Date']<=pd.to_datetime('2018-09-09')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-11-01'))&(Raw['Departure Date']<=pd.to_datetime('2018-11-04')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2018-12-22'))&(Raw['Departure Date']<=pd.to_datetime('2019-01-06')),'SEASON']='L'

Raw.loc[(Raw['Sched Airport Depart']=='LCY')&(Raw['Sched Airport Arriva']=='LUX')\
&(Raw['Departure Date']>=pd.to_datetime('2019-04-06'))&(Raw['Departure Date']<=pd.to_datetime('2019-04-22')),'SEASON']='L'

In [572]:
Raw=Raw[['Departure Date','Route','Flight Number','Direction','SEASON','LF_actual','Revenue_actual']]

#Raw['Year']=Raw['Departure Date'].apply(lambda x: x.year)
Raw['Year']='2017'
Raw['Year']=Raw['Departure Date']\
.apply(lambda x: '2017-2018' if (x>=pd.to_datetime('2017-04-01'))&(x<=pd.to_datetime('2018-03-31'))\
else ('2018-2019' if (x>=pd.to_datetime('2018-04-01'))&(x<=pd.to_datetime('2019-03-31')) else 'out'))
Raw=Raw.loc[Raw['Year'].apply(lambda x: (x=='2017-2018')|(x=='2018-2019')),:]

Raw['Month']=Raw['Departure Date'].apply(lambda x: x.month)
Raw['Weekday']=Raw['Departure Date'].apply(lambda x: x.weekday())

Raw=Raw[[x for x in Raw.columns if 'LF' not in x and 'Revenue' not in x]+[x for x in Raw.columns if 'LF' in x or 'Revenue' in x]]

Raw['Departure Date']=Raw['Departure Date'].apply(lambda x: str(x).split(' ')[0])
Raw['Standardized Departure Date']=Raw['Departure Date'].apply(lambda x: '2019-'+x[5:])

In [573]:
Bcg=pd.read_csv('Spoil upside calculation database with new RPK.csv',sep=';',skiprows=1)

Bcg['Route']='LUX'+'-'+Bcg['leg']

Bcg=Bcg.loc[((Bcg['leg']=='CPH')|(Bcg['leg']=='LCY'))&(Bcg['uplift_population']=='YES'),:]
Bcg.index=range(Bcg.shape[0])

Bcg=Bcg[['Departure Date','Route','Flight Number','SEASON','Sum_ASK','Sum_RPK','New RPK','Sum_Revenues','New Revenue']]

Bcg['LF_old']=Bcg['Sum_RPK']/Bcg['Sum_ASK']
Bcg['LF_new']=Bcg['New RPK']/Bcg['Sum_ASK']

Bcg=Bcg[[x for x in Bcg.columns if 'ASK' not in x and 'RPK' not in x]]

Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: pd.to_datetime(x.split('/')[2]+'-'+x.split('/')[1]+'-'+x.split('/')[0]))

Bcg['Month']=Bcg['Departure Date'].apply(lambda x: x.month)
Bcg['Weekday']=Bcg['Departure Date'].apply(lambda x: x.weekday())

Bcg=Bcg[[x for x in Bcg.columns if 'LF' not in x]+[x for x in Bcg.columns if 'LF' in x]]

Bcg['Sum_Revenues']=Bcg['Sum_Revenues'].astype('float')
Bcg['New Revenue']=Bcg['New Revenue'].astype('float')

In [574]:
Bcg['Departure Date']=Bcg['Departure Date'].apply(lambda x: str(x).split(' ')[0])
Bcg['Standardized Departure Date']=Bcg['Departure Date'].apply(lambda x: '2019-'+x[5:])

In [575]:
Bcg=Bcg.merge(Raw[cluster_variables+['Standardized Departure Date','LF_actual','Revenue_actual']],on=cluster_variables,how='left')

In [576]:
Bcg=Bcg.dropna(how='any')

In [577]:
Bcg['Standardized Departure Date_x']=Bcg['Standardized Departure Date_x'].apply(lambda x: pd.to_datetime(x))
Bcg['Standardized Departure Date_y']=Bcg['Standardized Departure Date_y'].apply(lambda x: pd.to_datetime(x))

In [578]:
Bcg['Date_Diff']=(Bcg['Standardized Departure Date_x']-Bcg['Standardized Departure Date_y']).apply(lambda x: abs(x.days))

In [579]:
Bcg['Date_Diff_min']=Bcg.groupby(['Standardized Departure Date_x','Flight Number','Weekday','SEASON'])['Date_Diff'].transform('min')

In [580]:
Bcg=Bcg.loc[(Bcg['Date_Diff']==Bcg['Date_Diff_min'])&(Bcg['Date_Diff']<=7),:]

In [581]:
# GROUPINGS

Bcg_Mean=Bcg.groupby(cluster_variables)['LF_old'].mean().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].mean().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['LF_actual'].mean().reset_index(),on=cluster_variables)

In [582]:
sum(Bcg_Mean['LF_actual']>Bcg_Mean['LF_old'])/Bcg_Mean.shape[0]

0.75

In [583]:
sum(Bcg_Mean['LF_actual']>Bcg_Mean['LF_new'])/Bcg_Mean.shape[0]

0.125

In [584]:
Bcg_Mean

,Route,Flight Number,Weekday,SEASON,LF_old,LF_new,LF_actual
0,LUX-CPH,LG-5431,2,M,0.500000,0.700000,0.602700
1,LUX-CPH,LG-5433,0,M,0.404040,0.946276,0.598167
2,LUX-CPH,LG-5433,1,L,0.485714,0.800000,0.616400
3,LUX-CPH,LG-5433,1,M,0.389556,0.725176,0.322987
4,LUX-CPH,LG-5433,2,H,0.471429,0.542857,0.647900
5,LUX-CPH,LG-5433,2,L,0.484848,0.702027,0.534200
6,LUX-CPH,LG-5433,2,M,0.386889,0.665263,0.347629
7,LUX-CPH,LG-5433,3,M,0.433024,0.772304,0.488229
8,LUX-CPH,LG-5433,4,L,0.363636,0.507576,0.465800
9,LUX-CPH,LG-5433,4,M,0.459596,1.009087,0.511400


In [585]:
Bcg_Mean=Bcg.groupby(cluster_variables)['Sum_Revenues'].mean().reset_index()\
.merge(Bcg.groupby(cluster_variables)['New Revenue'].mean().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['Revenue_actual'].mean().reset_index(),on=cluster_variables)

In [586]:
sum(Bcg_Mean['Revenue_actual']>Bcg_Mean['Sum_Revenues'])/Bcg_Mean.shape[0]

0.75

In [587]:
sum(Bcg_Mean['Revenue_actual']>Bcg_Mean['New Revenue'])/Bcg_Mean.shape[0]

0.1875

In [588]:
Bcg_Mean

,Route,Flight Number,Weekday,SEASON,Sum_Revenues,New Revenue,Revenue_actual
0,LUX-CPH,LG-5431,2,M,2427.000000,2646.000000,3878.000000
1,LUX-CPH,LG-5433,0,M,2616.000000,5579.000000,5538.666667
2,LUX-CPH,LG-5433,1,L,2678.000000,3744.000000,3608.000000
3,LUX-CPH,LG-5433,1,M,2637.750000,4067.375000,2598.375000
4,LUX-CPH,LG-5433,2,H,2622.000000,3312.000000,5456.000000
5,LUX-CPH,LG-5433,2,L,3513.000000,4136.000000,2933.000000
6,LUX-CPH,LG-5433,2,M,2523.571429,3904.571429,2853.571429
7,LUX-CPH,LG-5433,3,M,2671.571429,4102.000000,3341.571429
8,LUX-CPH,LG-5433,4,L,4464.000000,9329.000000,2403.000000
9,LUX-CPH,LG-5433,4,M,2184.666667,5030.000000,3251.666667


In [161]:
# GROUPINGS

Bcg_Mean=Bcg.groupby(cluster_variables)['LF_old'].mean().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].mean().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['Sum_Revenues'].mean().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['New Revenue'].mean().reset_index(),on=cluster_variables)\
.rename(columns={'LF_old': 'LF_old_mean', 'LF_new': 'LF_new_mean', 'Sum_Revenues': 'Sum_Revenues_mean', 'New Revenue': 'New Revenue_mean'})

Bcg_Var=Bcg.groupby(cluster_variables)['LF_old'].var().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].var().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['Sum_Revenues'].var().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['New Revenue'].var().reset_index(),on=cluster_variables)\
.rename(columns={'LF_old': 'LF_old_var', 'LF_new': 'LF_new_var', 'Sum_Revenues': 'Sum_Revenues_var', 'New Revenue': 'New Revenue_var'})

Bcg_N=Bcg.groupby(cluster_variables)['LF_old'].size().reset_index()\
.merge(Bcg.groupby(cluster_variables)['LF_new'].size().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['Sum_Revenues'].size().reset_index(),on=cluster_variables)\
.merge(Bcg.groupby(cluster_variables)['New Revenue'].size().reset_index(),on=cluster_variables)\
.rename(columns={'LF_old': 'LF_old_size', 'LF_new': 'LF_new_size', 'Sum_Revenues': 'Sum_Revenues_size', 'New Revenue': 'New Revenue_size'})

In [34]:
replace_value_old=np.mean(Bcg_Var['LF_old_var'])
replace_value_old2=np.mean(Bcg_Var['Sum_Revenues_var'])
replace_value_new=np.mean(Bcg_Var['LF_new_var'])
replace_value_new2=np.mean(Bcg_Var['New Revenue_var'])

Bcg_Var['LF_old_var']=Bcg_Var['LF_old_var'].apply(lambda x: replace_value_old if pd.isnull(x) or x==0 else x)
Bcg_Var['Sum_Revenues_var']=Bcg_Var['Sum_Revenues_var'].apply(lambda x: replace_value_old2 if pd.isnull(x) or x==0 else x)
Bcg_Var['LF_new_var']=Bcg_Var['LF_new_var'].apply(lambda x: replace_value_new if pd.isnull(x) or x==0 else x)
Bcg_Var['New Revenue_var']=Bcg_Var['New Revenue_var'].apply(lambda x: replace_value_new2 if pd.isnull(x) or x==0 else x)

In [35]:
Data=Bcg_Mean.merge(Bcg_Var,on=cluster_variables,how='left')\
.merge(Bcg_N,on=cluster_variables,how='left')\
.merge(Raw_Mean,on=cluster_variables,how='left')\
.merge(Raw_Var,on=cluster_variables,how='left')\
.merge(Raw_N,on=cluster_variables,how='left')

In [36]:
Data

,Route,Flight Number,Weekday,SEASON,LF_old_mean,LF_new_mean,Sum_Revenues_mean,New Revenue_mean,LF_old_var,LF_new_var,...,LF_old_size,LF_new_size,Sum_Revenues_size,New Revenue_size,LF_actual_mean,Revenue_actual_mean,LF_actual_var,Revenue_actual_var,LF_actual_n,Revenue_actual_n
0,LUX-CPH,LG-5431,2,M,0.500000,0.700000,2427.000000,2646.000000,0.004658,0.004770,...,1,1,1,1,0.369320,2562.800000,0.000431,4.331457e+05,5.0,5.0
1,LUX-CPH,LG-5431,3,H,0.457143,0.457143,3213.000000,3213.000000,0.004658,0.004770,...,1,1,1,1,0.479500,3225.000000,0.009748,2.406807e+06,1.0,1.0
2,LUX-CPH,LG-5432,2,M,0.500000,0.464295,2647.000000,2507.000000,0.004658,0.004770,...,1,1,1,1,NaN,NaN,NaN,NaN,NaN,NaN
3,LUX-CPH,LG-5433,0,M,0.404040,0.946276,2616.000000,5579.000000,0.003290,0.004770,...,3,3,3,3,0.407558,3599.916667,0.004778,8.503632e+05,12.0,12.0
4,LUX-CPH,LG-5433,1,L,0.477706,0.824242,2945.500000,3637.000000,0.000128,0.001175,...,2,2,2,2,0.321950,2505.500000,0.000845,2.429045e+05,2.0,2.0
5,LUX-CPH,LG-5433,1,M,0.389556,0.725176,2637.750000,4067.375000,0.003824,0.000480,...,8,8,8,8,0.261064,2036.272727,0.008790,5.807050e+05,11.0,11.0
6,LUX-CPH,LG-5433,2,H,0.471429,0.542857,2622.000000,3312.000000,0.004658,0.004770,...,1,1,1,1,0.647900,5456.000000,0.009748,2.406807e+06,1.0,1.0
7,LUX-CPH,LG-5433,2,L,0.484848,0.702027,3513.000000,4136.000000,0.004658,0.004770,...,1,1,1,1,0.376750,2552.500000,0.002346,3.354050e+04,2.0,2.0
8,LUX-CPH,LG-5433,2,M,0.386889,0.665263,2523.571429,3904.571429,0.001263,0.000356,...,7,7,7,7,0.272210,2120.400000,0.003093,1.030898e+05,10.0,10.0
9,LUX-CPH,LG-5433,3,M,0.433024,0.772304,2671.571429,4102.000000,0.007427,0.000479,...,7,7,7,7,0.324450,2444.000000,0.002468,4.800740e+05,10.0,10.0


In [37]:
Data['t_old_lf']=Data[['LF_actual_mean','LF_actual_var','LF_actual_n','LF_old_mean','LF_old_var','LF_old_size']]\
.apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1)
Data['t_old_rev']=Data[['Revenue_actual_mean','Revenue_actual_var','Revenue_actual_n','Sum_Revenues_mean','Sum_Revenues_var','Sum_Revenues_size']]\
.apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1)
Data['t_new_lf']=Data[['LF_actual_mean','LF_actual_var','LF_actual_n','LF_new_mean','LF_new_var','LF_new_size']]\
.apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1)
Data['t_new_rev']=Data[['Revenue_actual_mean','Revenue_actual_var','Revenue_actual_n','New Revenue_mean','New Revenue_var','New Revenue_size']]\
.apply(lambda x: (x[0]-x[3])/np.sqrt((x[1]/x[2])+(x[4]/x[5])),axis=1)

Data['Z_larger']=norm.ppf(0.95)
Data['Z_smaller']=-norm.ppf(0.95)

#Data=Data[cluster_variables+['t_old','t_new','Z_larger','Z_smaller']].sort_values(by=['Flight Number','Weekday'])

Data['decision_lf_old']=Data[['t_old_lf','Z_larger','Z_smaller']].apply(lambda x: 'Up' if x[0]>x[1] else ('Down' if x[0]<x[2] else 'Constant'),axis=1)
Data['decision_rev_old']=Data[['t_old_rev','Z_larger','Z_smaller']].apply(lambda x: 'Up' if x[0]>x[1] else ('Down' if x[0]<x[2] else 'Constant'),axis=1)
Data['decision_lf_new']=Data[['t_new_lf','Z_larger','Z_smaller']].apply(lambda x: 'Up' if x[0]>x[1] else ('Down' if x[0]<x[2] else 'Constant'),axis=1)
Data['decision_rev_new']=Data[['t_new_rev','Z_larger','Z_smaller']].apply(lambda x: 'Up' if x[0]>x[1] else ('Down' if x[0]<x[2] else 'Constant'),axis=1)

# Data['decision_new']=Data[['LF_old_mean','LF_new_mean','decision_old','decision_new']]\
# .apply(lambda x: x[2] if x[0]==x[1] else x[3],axis=1)

# Data=Data[cluster_variables+['LF_actual_mean','LF_old_mean','LF_new_mean','decision_old','decision_new']]\
# .sort_values(by=cluster_variables)

In [38]:
Data=Data.rename(columns={'LF_old_mean': 'LF_Data2017-2018', 'LF_new_mean': 'LF_BCG_Target', 'Sum_Revenues_mean': 'Revenue_Data2017-2018',
'New Revenue_mean': 'Revenue_BCG_Target', 'LF_actual_mean': 'LF_Data2018-2019', 'Revenue_actual_mean': 'Revenue_Data2018-2019',
'decision_lf_old': 'LF_versusData2017-2018', 'decision_rev_old': 'Revenue_versusData2017-2018', 
'decision_lf_new': 'LF_versusTarget', 'decision_rev_new': 'Revenue_versusTarget'})

In [39]:
Data.columns

Index(['Route', 'Flight Number', 'Weekday', 'SEASON', 'LF_Data2017-2018',
       'LF_BCG_Target', 'Revenue_Data2017-2018', 'Revenue_BCG_Target',
       'LF_old_var', 'LF_new_var', 'Sum_Revenues_var', 'New Revenue_var',
       'LF_old_size', 'LF_new_size', 'Sum_Revenues_size', 'New Revenue_size',
       'LF_Data2018-2019', 'Revenue_Data2018-2019', 'LF_actual_var',
       'Revenue_actual_var', 'LF_actual_n', 'Revenue_actual_n', 't_old_lf',
       't_old_rev', 't_new_lf', 't_new_rev', 'Z_larger', 'Z_smaller',
       'LF_versusData2017-2018', 'Revenue_versusData2017-2018',
       'LF_versusTarget', 'Revenue_versusTarget'],
      dtype='object')

In [40]:
Data['Sum_Data2017-2018']=Data['Revenue_Data2017-2018']*Data['LF_old_size']
Data['Sum_Data2018-2019']=Data['Revenue_Data2018-2019']*Data['LF_actual_n']

In [41]:
Data['Sum_Data2017-2018'].sum()

526837.0

In [42]:
Data['Sum_Data2018-2019'].sum()

759077.0

In [43]:
Data['LF_actual_n']

0      5.0
1      1.0
2      NaN
3     12.0
4      2.0
5     11.0
6      1.0
7      2.0
8     10.0
9     10.0
10     4.0
11    11.0
12     2.0
13     NaN
14     NaN
15     NaN
16     NaN
17     NaN
18     NaN
19     NaN
20     NaN
21     NaN
22     2.0
23    11.0
24     2.0
25    11.0
26     3.0
27     3.0
28    11.0
29     2.0
      ... 
56     NaN
57     NaN
58     NaN
59     NaN
60     NaN
61     NaN
62     NaN
63     NaN
64     NaN
65     NaN
66     NaN
67     NaN
68     NaN
69     NaN
70     NaN
71    14.0
72     NaN
73     NaN
74     NaN
75    13.0
76     NaN
77     NaN
78    14.0
79     NaN
80     NaN
81     NaN
82     NaN
83     7.0
84     1.0
85     NaN
Name: LF_actual_n, Length: 86, dtype: float64

In [44]:
Data=Data[['Route','Flight Number','Weekday','SEASON','LF_Data2017-2018','LF_BCG_Target','LF_Data2018-2019','LF_versusData2017-2018','LF_versusTarget',
'Revenue_Data2017-2018','Revenue_BCG_Target','Revenue_Data2018-2019','Revenue_versusData2017-2018','Revenue_versusTarget']]

In [45]:
Data['LF_Data2017-2018']=(Data['LF_Data2017-2018'].round(4)*100).apply(lambda x: str(x)+'%')
Data['LF_BCG_Target']=(Data['LF_BCG_Target'].round(4)*100).apply(lambda x: str(x)+'%')
Data['LF_Data2018-2019']=(Data['LF_Data2018-2019'].round(4)*100).apply(lambda x: str(x)+'%')

In [46]:
Weekday_map=DataFrame([[0,1,2,3,4,5,6],['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday']]).transpose()
Weekday_map.columns=['Weekday','Weekday_new']

In [47]:
Data=Data.merge(Weekday_map,on='Weekday')

In [48]:
Data['Weekday']=Data['Weekday_new']
Data=Data[[x for x in Data.columns if x!='Weekday_new']]

In [49]:
Data.to_csv('PythonExtract_Mean.csv',index=False)

In [50]:
Data

,Route,Flight Number,Weekday,SEASON,LF_Data2017-2018,LF_BCG_Target,LF_Data2018-2019,LF_versusData2017-2018,LF_versusTarget,Revenue_Data2017-2018,Revenue_BCG_Target,Revenue_Data2018-2019,Revenue_versusData2017-2018,Revenue_versusTarget
0,LUX-CPH,LG-5431,Wednesday,M,50.0%,70.0%,36.93%,Down,Down,2427.000000,2646.000000,2562.800000,Constant,Constant
1,LUX-CPH,LG-5432,Wednesday,M,50.0%,46.43%,nan%,Constant,Constant,2647.000000,2507.000000,NaN,Constant,Constant
2,LUX-CPH,LG-5433,Wednesday,H,47.14%,54.290000000000006%,64.79%,Constant,Constant,2622.000000,3312.000000,5456.000000,Up,Constant
3,LUX-CPH,LG-5433,Wednesday,L,48.480000000000004%,70.19999999999999%,37.68%,Constant,Down,3513.000000,4136.000000,2552.500000,Down,Constant
4,LUX-CPH,LG-5433,Wednesday,M,38.690000000000005%,66.53%,27.22%,Down,Down,2523.571429,3904.571429,2120.400000,Down,Down
5,LUX-CPH,LG-5434,Wednesday,M,40.17%,63.519999999999996%,nan%,Constant,Constant,2630.444444,4097.555556,NaN,Constant,Constant
6,LUX-CPH,LG-5436,Wednesday,H,53.03%,78.03%,nan%,Constant,Constant,3033.000000,5328.000000,NaN,Constant,Constant
7,LUX-CPH,LG-5436,Wednesday,M,45.45%,98.83%,nan%,Constant,Constant,2538.000000,6755.000000,NaN,Constant,Constant
8,LUX-LCY,LG-4591,Wednesday,H,41.160000000000004%,69.97%,52.11%,Constant,Down,3587.000000,5241.000000,1969.000000,Constant,Down
9,LUX-LCY,LG-4594,Wednesday,H,48.39%,89.14999999999999%,nan%,Constant,Constant,3410.000000,5379.000000,NaN,Constant,Constant
